### На любом языке программирования написать программу, реализующую разобранные на занятии критерии проверки гипотез о равенстве вероятностей и средних значений. В качестве ответа программа должна возвращать - значение статистики критерия, ответ на вопрос принимается или отвергается основная гипотеза на выбранном уровне значимости, Р-значение. Ссылку на код на гитхабе прислать в чат с преподавателем.

In [1]:
import re
import numpy as np
from scipy.stats import norm

def type_input():
    task_type = input('Укажите тип задачи: a) о равенстве вероятностей или b) о средних значениях:\t')
    while task_type not in ['a', 'b']:
        task_type = input('Укажите a или b:\t')
    hyp = input('Введите альтернативную гипотезу:\t')
    HA = re.findall('<|>|!=', hyp)[0]
    return task_type, HA

def trust_area(HA, t):
    alpha = float(input('Введите уровень значимости альфа:\t'))
    if HA == '<':
        ul = norm.ppf(alpha)
        ur = 100
        p_value = norm.cdf(t)
    elif HA == '>':
        ul = -100
        ur = norm.ppf(1 - alpha)
        p_value = 1 - norm.cdf(t)
    elif HA == '!=':
        ul = norm.ppf(alpha / 2)
        ur = norm.ppf(1-alpha / 2)
        p_value = np.min([2 * norm.cdf(t), 2 * (1 - norm.cdf(t))])
    return ul, ur, p_value, alpha

def possibility_solver():
    a1 = float(input('Введите количество благоприятных событий в варианте 1:\t'))
    n1 = float(input('Введите общее количество событий в варианте 1:\t'))
    a2 = float(input('Введите количество благоприятных событий в варианте 2:\t'))
    n2 = float(input('Введите общее количество событий в варианте 2:\t'))
    p0 = (a1 + a2) / (n1 + n2)
    p1 = a1 / n1
    p2 = a2 / n2
    t = (p1 - p2) / (p0 * (1 - p0) * (1 / n1 + 1 / n2)) ** 0.5
    return t

def series_solver():
    x = input('Введите серию 1 (разделитель между значениями - пробел, между десятми и целыми - точка):\t')
    y = input('Введите серию 2 (разделитель между значениями - пробел, между десятми и целыми - точка):\t')
    x = [float(i) for i in x.split()]
    y = [float(i) for i in y.split()]
    total = x.copy()
    total.extend(y)
    total.sort()
    total_dict = {}
    for i, j in enumerate(total):
        if j not in total_dict.keys():
            total_dict[j] = [i + 1]
        else:
            total_dict[j].append(i + 1)
    for key in total_dict.keys():
        total_dict[key] = np.mean(total_dict[key])
    # Нахождение критерия Вилкоксона как сумма рангов 2ой выборки
    w = 0
    for elem in y:
        w += total_dict[elem]
    # Расчет центрированного и нормированного критерия Вилкоксона
    m = len(x)
    n = len(y)
    E_w = 0.5 * n * (m + n + 1)
    D_w = (m * n * (m + n + 1))**2
    t = (w - E_w)/(D_w)**0.5
    return t
    
    
def check_hypothesis():
    task_type, HA = type_input()
    
    if task_type == 'a':
        t = possibility_solver()
    else:
        t = series_solver()
        
    ul, ur, p_value, alpha = trust_area(HA, t)
    
    if (t > ul) & (t < ur):
        print('Гипотеза H0 принимается на уровне значимости {}'.format(alpha))
    else:
        print('На уровне значимости {} гипотеза H0 не может быть принята - принимаем гипотезу HA'.format(alpha))
        
    print('Уровень значимости = {}, p-value = {}'.format(alpha, p_value))

### Задача 1
Перед президентскими выборами  в городах Курске и Владивостоке был проведен социологический опрос. Каждый респондент должен был ответить на вопрос: «За какого кандидата вы будете голосовать на выборах?». В Курске опросили 105 человек, из них 42 ответили, что будут голосовать за кандидата А, во Владивостоке опросили 195 человек, из которых 65 за А. Можно ли считать на уровне значимости 0,05, что уровни поддержки кандидата А в Курске и Владивостоке одинаковы?

<br> H0: pk = pv -уровни поддержки кандидата А в Курске и Владивостоке одинаковы
<br> HA: pk != pv -уровни поддержки кандидата А в Курске и Владивостоке разные

In [2]:
check_hypothesis()

Укажите тип задачи: a) о равенстве вероятностей или b) о средних значениях:	a
Введите альтернативную гипотезу:	!=
Введите количество благоприятных событий в варианте 1:	42
Введите общее количество событий в варианте 1:	105
Введите количество благоприятных событий в варианте 2:	65
Введите общее количество событий в варианте 2:	195
Введите уровень значимости альфа:	0.05
Гипотеза H0 принимается на уровне значимости 0.05
Уровень значимости = 0.05, p-value = 0.2502388245506466


### Задача 2.
Для изучения эффективности лекарства против аллергии обследовались две группы людей, предрасположенных к этому заболеванию. Результаты обследования следующие: среди принимавших лекарство заболело 3 человека, не заболело 172 человека; среди не  принимавших заболело 32 человека, не заболело 168. Указывают ли эти результаты на эффективность лекарства?

<br> H0: p1 = p2 -эффективность лекарства отсутствует
<br> HA: p1 < p2 -применение лекарства снижает уровень заболевания, лекарство эффективно

In [4]:
check_hypothesis()

Укажите тип задачи: a) о равенстве вероятностей или b) о средних значениях:	a
Введите альтернативную гипотезу:	<
Введите количество благоприятных событий в варианте 1:	3
Введите общее количество событий в варианте 1:	175
Введите количество благоприятных событий в варианте 2:	32
Введите общее количество событий в варианте 2:	168
Введите уровень значимости альфа:	0.05
На уровне значимости 0.05 гипотеза H0 не может быть принята - принимаем гипотезу HA
Уровень значимости = 0.05, p-value = 5.7448005995211975e-08


### Задача 3
Было проведено обследование 10 горожан и 9 жителей села примерно одного возраста. Получены следующие данные об уровне давления:
для горожан: 130, 110, 120, 140, 200, 130, 140, 170, 160, 140;
для селян: 120, 190, 130, 160, 150, 120, 110, 120, 200.
Свидетельствуют ли эти данные в пользу того, что горожане имеют в среднем более высокое давление чем жители сельской местности?

<br> H0: tetha = 0 -средние давления горожан (1ая выборка x) и селян (2ая выборака y) одинаковые
<br> HA: tetha < 0 -среднее давление селян ниже, а горожан выше

In [5]:
check_hypothesis()

Укажите тип задачи: a) о равенстве вероятностей или b) о средних значениях:	b
Введите альтернативную гипотезу:	<
Введите серию 1 (разделитель между значениями - пробел, между десятми и целыми - точка):	130 110 120 140 200 130 140 170 160 140
Введите серию 2 (разделитель между значениями - пробел, между десятми и целыми - точка):	120 190 130 160 150 120 110 120 200
Введите уровень значимости альфа:	0.05
Гипотеза H0 принимается на уровне значимости 0.05
Уровень значимости = 0.05, p-value = 0.49911346232876863


### Задача 4
Уровень гистамина в мокроте у 7 курильщиков, склонных к аллергии, составил в мг:
102,4	100,0	67,6	65,9	64,7	39,6	31,2
У 10 курильщиков не склонных к аллергии составил в мг:
48,1	45,5	41,7	35,4	29,1	18,9	58,3	68,8	71,3	94,3
Можно ли, основываясь на этих данных, считать с надёжностью 0,95 что уровень гистамина у склонных и не склонных к аллергии курильщиков значимо различается?

<br> H0: tetha = 0 -средние значения уровня гистамина у 2х выборок одинаковые
<br> HA: tetha != 0 -среднее значения уровня гистамина отличаются

In [2]:
check_hypothesis()

Укажите тип задачи: a) о равенстве вероятностей или b) о средних значениях:	b
Введите альтернативную гипотезу:	!=
Введите серию 1 (разделитель между значениями - пробел, между десятми и целыми - точка):	102.4 100.0 67.6 65.9 64.7 39.6 31.2
Введите серию 2 (разделитель между значениями - пробел, между десятми и целыми - точка):	48.1 45.5 41.7 35.4 29.1 18.9 58.3 68.8 71.3 94.3
Введите уровень значимости альфа:	0.05
Гипотеза H0 принимается на уровне значимости 0.05
Уровень значимости = 0.05, p-value = 0.9930344296168977
